In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import time
import os   # Das ist ein Syteminterface-Tool (um etwas im System zu machen. hier: eine Liste des Ordners anzeigen)
import re
from tqdm import tqdm

In [2]:
lst = os.listdir('pages/')
# wenn mans sortiert möchte, kann man noch sorted einfügen: 
lst = sorted(os.listdir('pages/'))

In [3]:
lst

['page.htm',
 'page0.htm',
 'page1.htm',
 'page10.htm',
 'page11.htm',
 'page12.htm',
 'page13.htm',
 'page14.htm',
 'page15.htm',
 'page16.htm',
 'page17.htm',
 'page18.htm',
 'page19.htm',
 'page2.htm',
 'page20.htm',
 'page21.htm',
 'page22.htm',
 'page23.htm',
 'page24.htm',
 'page25.htm',
 'page26.htm',
 'page27.htm',
 'page28.htm',
 'page29.htm',
 'page3.htm',
 'page30.htm',
 'page31.htm',
 'page32.htm',
 'page33.htm',
 'page34.htm',
 'page4.htm',
 'page5.htm',
 'page6.htm',
 'page7.htm',
 'page8.htm',
 'page9.htm']

In [4]:
file = open('pages/' + lst[0], 'r')
text = file.read()
h = BeautifulSoup(text, 'html.parser')

In [5]:
# Ein einzelnes Element aus dieser Liste anschauen:
daten = h.find_all('tr', {'class':'ng-scope'})
daten[0]
# Hier sehen wir: der Name verbirgt sich in einem div-Tag

<tr class="ng-scope" ng-repeat="row in $data"><!-- ngRepeat: col in $columns --><!-- ngIf: col.show() --><td class="ng-scope" data-title-text="business name" filter="$listCtrl.filters[col.name]" ng-if="col.show()" ng-repeat="col in $columns"><!-- ngIf: col.name === 'company' --><div class="company-name ng-scope" ng-attr-title="{{row.name}}" ng-if="col.name === 'company'" title='Bäckerei "gsund" AG'><a class="ng-binding" href="/en/search/entity/list/firm/1167884?name=b%C3%A4ckerei&amp;searchType=exact" ng-bind-html="highlight(row.name, $listCtrl.tableParams.filter(), 'name')" ng-class="{ 'strike-thru': row.status === 'GELOESCHT'}" ui-sref=".details({ ehraid: row.ehraid })">Bäckerei "gsund" AG</a></div><!-- end ngIf: col.name === 'company' --><!-- ngIf: col.name === 'legalSeat' --><!-- ngIf: col.getValue --></td><!-- end ngIf: col.show() --><!-- end ngRepeat: col in $columns --><!-- ngIf: col.show() --><td class="ng-scope" data-title-text="excerpt" filter="$listCtrl.filters[col.name]" ng

In [6]:
daten[0].find_all("div")

[<div class="company-name ng-scope" ng-attr-title="{{row.name}}" ng-if="col.name === 'company'" title='Bäckerei "gsund" AG'><a class="ng-binding" href="/en/search/entity/list/firm/1167884?name=b%C3%A4ckerei&amp;searchType=exact" ng-bind-html="highlight(row.name, $listCtrl.tableParams.filter(), 'name')" ng-class="{ 'strike-thru': row.status === 'GELOESCHT'}" ui-sref=".details({ ehraid: row.ehraid })">Bäckerei "gsund" AG</a></div>,
 <div class="ng-binding ng-scope" ng-bind-html="col.getValue(this, row)" ng-if="col.getValue"><a href="https://sz.chregister.ch/cr-portal/auszug/zefix.xhtml?uid=CHE-240.641.011&amp;lang=de" style="margin: 0em 1em;" target="_blank" title="excerpt (Web)"><span class="icon icon--external"></span></a> </div>,
 <div class="ng-binding ng-scope" ng-bind-html="col.getValue(this, row)" ng-if="col.getValue"><span class="no-wrap">CHE-240.641.011<a class="uid-link" href="https://www.uid.admin.ch/Detail.aspx?uid_id=CHE240641011" target="_blank"><span class="icon icon--exte

In [7]:
name = daten[0].find_all("div")[0].text
ort = daten[0].find_all("div")[4].text
ID = daten[0].find_all("div")[2].text
URL = daten[0].find_all("a")[1]["href"]

datensatz = {"Name" : name, "Ort" : ort, "ID-Nr." : ID, "Website" : URL}

print(datensatz)

{'Name': 'Bäckerei "gsund" AG', 'Ort': 'Feusisberg', 'ID-Nr.': 'CHE-240.641.011', 'Website': 'https://sz.chregister.ch/cr-portal/auszug/zefix.xhtml?uid=CHE-240.641.011&lang=de'}


In [8]:
df = pd.DataFrame([datensatz])
df

,Name,Ort,ID-Nr.,Website
0,"Bäckerei ""gsund"" AG",Feusisberg,CHE-240.641.011,https://sz.chregister.ch/cr-portal/auszug/zefi...


**Jetzt loopen wir das Ganze:**

In [15]:
datenliste = []

daten = h.find_all('tr', {'class':'ng-scope'})

for zeile in daten:
    
    name = zeile.find_all("div")[0].text
    ort = zeile.find_all("div")[4].text
    ID = zeile.find_all("div")[2].text
    URL = zeile.find_all("a")[1]["href"]

    datensatz = {"Name" : name, 
                 "Ort" : ort, 
                 "ID-Nr." : ID, 
                 "Website" : URL}
    
    datenliste.append(datensatz)
    
df = pd.DataFrame(datenliste)
df

,Name,Ort,ID-Nr.,Website
0,"Bäckerei ""gsund"" AG",Feusisberg,CHE-240.641.011,https://sz.chregister.ch/cr-portal/auszug/zefi...
1,Bäckerei & Catering M Janicijevic,Zürich,CHE-114.960.579,https://zh.chregister.ch/cr-portal/auszug/zefi...
2,"Bäckerei & Konditorei, Biner & Co.",Zermatt,CHE-102.073.772,https://vo.chregister.ch/cr-portal/auszug/zefi...
3,Bäckerei + Party-Service Sandra Bircher,Wölflinswil,CHE-115.910.495,https://ag.chregister.ch/cr-portal/auszug/zefi...
4,Bäckerei - Konditorei - Café Gut,Willisau,CHE-112.034.070,https://lu.chregister.ch/cr-portal/auszug/zefi...
5,Bäckerei - Konditorei - Lebensmittel Leuenberger,Erlinsbach (SO),CHE-107.195.164,https://so.chregister.ch/cr-portal/auszug/zefi...
6,Bäckerei - Konditorei - Lebensmittel Oester,Adelboden,CHE-109.306.718,https://be.chregister.ch/cr-portal/auszug/zefi...
7,"Bäckerei - Konditorei Adula, Georg Schmid und ...",Tujetsch,CHE-438.593.065,https://gr.chregister.ch/cr-portal/auszug/zefi...
8,Bäckerei - Konditorei D. Pfister,Uetendorf,CHE-116.198.463,https://be.chregister.ch/cr-portal/auszug/zefi...
9,Bäckerei - Konditorei Eichenberger AG,Langnau im Emmental,CHE-313.666.724,https://be.chregister.ch/cr-portal/auszug/zefi...


Bei **Loops in Loops** packt man den Loop am besten in eine **neu definierte Funktion**, damits übersichtlicher bleibt:

In [50]:
def extractbakeries(elem):
    
    datenliste = []

    daten = elem.find_all('tr', {'class':'ng-scope'})

    for zeile in daten:
    
        name = zeile.find_all("div")[0].text
        ort = zeile.find_all("div")[4].text
        ID = zeile.find_all("div")[2].text
        URL = zeile.find_all("a")[1]["href"]

        datensatz = {"Name" : name, 
                     "Ort" : ort, 
                     "ID-Nr." : ID, 
                     "Website" : URL}
    
        datenliste.append(datensatz)
    
    return(datenliste)

Jetzt können wir mit dieser Funktion ganez elegant für jede Seite (h) das Resultat anzeigen lassen (indem wir den Seitenwert anpassen:

In [51]:
extractbakeries(h)

[{'Name': 'BÄCKEREI - KONDITOREI FÄSSLER',
  'Ort': 'Rüte',
  'ID-Nr.': 'CHE-115.132.898',
  'Website': 'https://ai.chregister.ch/cr-portal/auszug/zefix.xhtml?uid=CHE-115.132.898&lang=de'},
 {'Name': 'Bäckerei - Konditorei Furter, Nachfolger Mario Monaco-Oesch',
  'Ort': 'Aarau',
  'ID-Nr.': 'CHE-109.386.806',
  'Website': 'https://ag.chregister.ch/cr-portal/auszug/zefix.xhtml?uid=CHE-109.386.806&lang=de'},
 {'Name': 'Bäckerei - Konditorei Gübeli',
  'Ort': 'St. Stephan',
  'ID-Nr.': 'CHE-114.920.189',
  'Website': 'https://be.chregister.ch/cr-portal/auszug/zefix.xhtml?uid=CHE-114.920.189&lang=de'},
 {'Name': 'Bäckerei - Konditorei Hermann',
  'Ort': 'Muhen',
  'ID-Nr.': 'CHE-107.066.035',
  'Website': 'https://ag.chregister.ch/cr-portal/auszug/zefix.xhtml?uid=CHE-107.066.035&lang=de'},
 {'Name': 'Bäckerei - Konditorei Jean-Pierre Grindat',
  'Ort': 'Orpund',
  'ID-Nr.': 'CHE-106.154.151',
  'Website': 'https://be.chregister.ch/cr-portal/auszug/zefix.xhtml?uid=CHE-106.154.151&lang=de'}

also so:

In [55]:
file = open('pages/' + lst[2], 'r')   # hier habe ich die Seite verändert
text = file.read()
h = BeautifulSoup(text, 'html.parser')

df = pd.DataFrame(extractbakeries(h))
df

,Name,Ort,ID-Nr.,Website
0,Bäckerei / Konditorei Ganz,Langenthal,CHE-204.892.391,https://be.chregister.ch/cr-portal/auszug/zefi...
1,Bäckerei / Konditorei Leutwyler GmbH,Zofingen,CHE-346.380.826,https://ag.chregister.ch/cr-portal/auszug/zefi...
2,Bäckerei / Konditorei Rüegger GmbH,Langnau im Emmental,CHE-105.022.067,https://be.chregister.ch/cr-portal/auszug/zefi...
3,Bäckerei A. Mathieu-Zumstein AG,Leuk,CHE-105.756.488,https://vo.chregister.ch/cr-portal/auszug/zefi...
4,Bäckerei Abderhalden,Herisau,CHE-107.351.748,https://ar.chregister.ch/cr-portal/auszug/zefi...
5,Bäckerei Abderhalden AG Wattwil,Wattwil,CHE-107.254.772,https://sg.chregister.ch/cr-portal/auszug/zefi...
6,Bäckerei Aebersold GmbH,Murten,CHE-100.206.039,https://adm.appls.fr.ch/hrcmatic/hrcintapp/ext...
7,Bäckerei Allenspach GmbH,Fischingen,CHE-265.077.575,https://tg.chregister.ch/cr-portal/auszug/zefi...
8,Bäckerei alpascha GmbH,Münchenstein,CHE-198.680.660,https://bl.chregister.ch/cr-portal/auszug/zefi...
9,Bäckerei Amrein AG,Büron,CHE-264.368.722,https://lu.chregister.ch/cr-portal/auszug/zefi...


Das ganze können wir jetzt loopen (und alles zusammenfügen)

In [56]:
final_list = []

for page in lst:
    file = open("pages/"+page, 'r')   # hier habe ich die Seite verändert
    text = file.read()
    h = BeautifulSoup(text, 'html.parser')

    extractbakeries(h)
    
    final_list.append(extractbakeries(h))

In [57]:
df = pd.DataFrame(final_list)
df

,0,1,2,3,4,5,6,7,8,9
0,"{'Name': 'Bäckerei ""gsund"" AG', 'Ort': 'Feusis...","{'Name': 'Bäckerei & Catering M Janicijevic', ...","{'Name': 'Bäckerei & Konditorei, Biner & Co.',...",{'Name': 'Bäckerei + Party-Service Sandra Birc...,"{'Name': 'Bäckerei - Konditorei - Café Gut', '...",{'Name': 'Bäckerei - Konditorei - Lebensmittel...,{'Name': 'Bäckerei - Konditorei - Lebensmittel...,"{'Name': 'Bäckerei - Konditorei Adula, Georg S...","{'Name': 'Bäckerei - Konditorei D. Pfister', '...",{'Name': 'Bäckerei - Konditorei Eichenberger A...
1,"{'Name': 'BÄCKEREI - KONDITOREI FÄSSLER', 'Ort...","{'Name': 'Bäckerei - Konditorei Furter, Nachfo...","{'Name': 'Bäckerei - Konditorei Gübeli', 'Ort'...","{'Name': 'Bäckerei - Konditorei Hermann', 'Ort...",{'Name': 'Bäckerei - Konditorei Jean-Pierre Gr...,"{'Name': 'Bäckerei - Konditorei Leuenberger', ...","{'Name': 'Bäckerei - Konditorei Maier GmbH', '...","{'Name': 'Bäckerei - Konditorei Märchy GmbH', ...","{'Name': 'Bäckerei - Konditorei Not Clalüna', ...",{'Name': 'Bäckerei - Konditorei Unternährer & ...
2,"{'Name': 'Bäckerei / Konditorei Ganz', 'Ort': ...",{'Name': 'Bäckerei / Konditorei Leutwyler GmbH...,"{'Name': 'Bäckerei / Konditorei Rüegger GmbH',...","{'Name': 'Bäckerei A. Mathieu-Zumstein AG', 'O...","{'Name': 'Bäckerei Abderhalden', 'Ort': 'Heris...","{'Name': 'Bäckerei Abderhalden AG Wattwil', 'O...","{'Name': 'Bäckerei Aebersold GmbH', 'Ort': 'Mu...","{'Name': 'Bäckerei Allenspach GmbH', 'Ort': 'F...","{'Name': 'Bäckerei alpascha GmbH', 'Ort': 'Mün...","{'Name': 'Bäckerei Amrein AG', 'Ort': 'Büron',..."
3,"{'Name': 'Bäckerei Konditorei Hackebeil', 'Ort...",{'Name': 'Bäckerei Konditorei Hess AG in Liqui...,"{'Name': 'Bäckerei Konditorei Hunger AG', 'Ort...","{'Name': 'Bäckerei Konditorei Hürlimann AG', '...",{'Name': 'Bäckerei Konditorei Lebensmittel Urs...,{'Name': 'Bäckerei Konditorei Mohler Inhaber R...,"{'Name': 'Bäckerei Konditorei Odermatt GmbH', ...","{'Name': 'Bäckerei Konditorei P. Scheiwiller',...","{'Name': 'Bäckerei Konditorei Schenk', 'Ort': ...","{'Name': 'Bäckerei Konditorei Siegrist GmbH', ..."
4,{'Name': 'Bäckerei Konditorei Singaraja Sivaji...,{'Name': 'Bäckerei Konditorei Staudenmann GmbH...,"{'Name': 'Bäckerei Konditorei Tanner', 'Ort': ...","{'Name': 'Bäckerei Konditorei Teixeira GmbH', ...","{'Name': 'Bäckerei Konditorei Zeller AG', 'Ort...","{'Name': 'Bäckerei Krähenbühl GmbH', 'Ort': 'K...","{'Name': 'Bäckerei Kronig Charly', 'Ort': 'Mör...","{'Name': 'Bäckerei Krummenacher AG', 'Ort': 'G...","{'Name': 'Bäckerei Kunz GmbH', 'Ort': 'Altnau'...","{'Name': 'Bäckerei Künzler GmbH', 'Ort': 'St. ..."
5,"{'Name': 'Bäckerei La Panetteria GmbH', 'Ort':...","{'Name': 'Bäckerei Lanz AG', 'Ort': 'Bern', 'I...","{'Name': 'Bäckerei Laube AG', 'Ort': 'Derendin...","{'Name': 'Bäckerei Lötscher AG', 'Ort': 'Escho...","{'Name': 'Bäckerei M. Leuenberger', 'Ort': 'Bu...",{'Name': 'Bäckerei Maria del Carmen Nieto GmbH...,"{'Name': 'Bäckerei Markus Ryser', 'Ort': 'Hilt...","{'Name': 'Bäckerei Marty Conditorei GmbH', 'Or...","{'Name': 'Bäckerei Meyer GmbH', 'Ort': 'Bern',...","{'Name': 'Bäckerei Michel GmbH', 'Ort': 'Adelb..."
6,"{'Name': 'Bäckerei Mödinger GmbH', 'Ort': 'Ber...","{'Name': 'Bäckerei Montero', 'Ort': 'Wünnewil-...","{'Name': 'Bäckerei Moor gmbh', 'Ort': 'Niederw...","{'Name': 'Bäckerei Moser', 'Ort': 'Heimberg', ...","{'Name': 'Bäckerei Motzer AG', 'Ort': 'Thal', ...","{'Name': 'Bäckerei Müller AG', 'Ort': 'Solothu...","{'Name': 'Bäckerei Mürner, Inhaber Sonja + Rol...","{'Name': 'Bäckerei Nellie AG', 'Ort': 'Heiden'...","{'Name': 'Bäckerei Neuenschwander AG', 'Ort': ...","{'Name': 'Bäckerei Neuhof GmbH', 'Ort': 'Schli..."
7,"{'Name': 'Bäckerei Nussbaumer AG', 'Ort': 'Cha...","{'Name': 'Bäckerei Nyffenegger GmbH', 'Ort': '...","{'Name': 'Bäckerei Oehler', 'Ort': 'Dittingen'...","{'Name': 'Bäckerei Patric Inniger', 'Ort': 'Th...","{'Name': 'Bäckerei Pfiffner AG', 'Ort': 'Mels'...","{'Name': 'Bäckerei Plüss im Limmatfeld GmbH', ...","{'Name': 'Bäckerei R. Sutter', 'Or